# XGBoostのみで予測
## scoreはまだいい方だが、stackingと比べると下がる
## 特徴量エンジニアリングが効いている感じ

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8")) #check the files available in the directory

# データクレンジニング

In [1]:
train=pd.read_csv('input/train.csv')
test=pd.read_csv('input/test.csv')

train_id=train['Id']
test_id=test['Id']
# inplace=Trueにすれば変更を元のデータに反映させることが可能
train.drop("Id",axis=1,inplace=True)
test.drop("Id",axis=1,inplace=True)

train=train.drop(train[(train['GrLivArea']>4000) &(train['SalePrice']<300000)].index)
train['SalePrice']=np.log1p(train['SalePrice'])

y_train=train.SalePrice.values
print(y_train.shape)
#train['SalePrice'].values

all_data=pd.concat([train,test]).reset_index(drop=True)
all_data.drop(['SalePrice'],axis=1,inplace=True)

all_data_na=(all_data.isnull().sum()/len(all_data))*100
all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)

all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")

# 中央値を欠損値に入れる groupbyでneghborhoodごとの中央値を取っている
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
    
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
    
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

all_data = all_data.drop(['Utilities'], axis=1)

train.groupby("Neighborhood").groups

print(all_data['Functional'].describe())
all_data["Functional"] = all_data["Functional"].fillna("Typ")

print(all_data['Electrical'].describe())
# mode()で最頻出を取得
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])

all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])

all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()


# .apply で関数を適用する
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)

# .astype でデータ型を変える
all_data['OverallCond'] = all_data['OverallCond'].astype(str)

all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

from sklearn.preprocessing import LabelEncoder
pd.options.display.max_columns = None
all_data.info()
# 特定のデータ型のみ取り出しselect_dtypes
all_data.select_dtypes(include=object)

cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

for c in cols:
    lbl=LabelEncoder()
    lbl.fit(list(all_data[c].values))
    all_data[c]=lbl.transform(list(all_data[c].values))
print('Shape all_data: {}'.format(all_data.shape))

# 新しい特徴量を定義
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

# 数字の特徴量のカラムを取得
numeric_feats=all_data.dtypes[all_data.dtypes != 'object'].index

# 歪度を取得
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

skewness=skewness[abs(skewness)>0.75]
from scipy.special import boxcox1p

skewed_features=skewness.index
lam=0.15

for feat in skewed_features:
    all_data[feat]=boxcox1p(all_data[feat],lam)
    
all_data=pd.get_dummies(all_data)
print(all_data.shape)

ntrain=train.shape[0]
print(ntrain)
ntest=test.shape[0]
print(ntest)
train=all_data[:ntrain]
test=all_data[ntrain:]

X_train=train

NameError: name 'pd' is not defined

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import xgboost as xgb

def rmse_cv(model):
    rmse=np.sqrt(-cross_val_score(model,X_train,y_train,scoring="neg_mean_squared_error",cv=5))
    return rmse

model=xgb.XGBRegressor(n_estimators=360,max_depth=2,learning_rate=0.1)
#model.fit(X_train,y_train)
rmse_xgb=rmse_cv(model)
print(rmse_xgb.mean())

[16:16:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:16:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:16:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:16:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:16:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.11919002671941781


In [7]:
model.fit(X_train,y_train)
predictions=model.predict(test)

[16:17:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [17]:
sub=pd.DataFrame({"Id":test_id,"SalePrice":np.expm1(predictions)})

In [19]:
sub.to_csv("xgboost_model.csv",index=False)

In [20]:
results=pd.read_csv("xgboost_model.csv")

In [21]:
results

,Id,SalePrice
0,1461,122430.180
1,1462,161523.110
2,1463,189742.060
3,1464,193451.030
4,1465,190246.520
5,1466,170779.730
6,1467,174020.080
7,1468,163842.690
8,1469,188531.030
9,1470,124028.220
